In [1]:
from pprint import pprint
import pickle
import numpy as np
import tensorflow as tf
import sys
import time
sys.path.append("../sketchformer")
sys.path.append("../src")


In [2]:
with open('../outputs/sample.pkl', 'rb') as f:
    sample = pickle.load(f)
print(len(sample), len(sample[0]), len(sample[0][0]['sketch']))
pprint(sample[0])


16 22 56
[{'label': 'tree',
  'position': [715, 390],
  'sketch': [[-4, 52, 0],
             [-30, 88, 0],
             [-28, 54, 1],
             [143, -175, 0],
             [-3, 5, 0],
             [-7, 71, 0],
             [0, 31, 0],
             [3, 15, 0],
             [12, 29, 0],
             [42, 46, 1],
             [-123, -206, 0],
             [-32, -1, 0],
             [-27, -5, 0],
             [-10, -3, 0],
             [-16, -11, 0],
             [-11, -23, 0],
             [0, -48, 0],
             [24, -29, 0],
             [10, -7, 0],
             [17, -5, 0],
             [7, 7, 0],
             [-1, 26, 0],
             [-5, 4, 0],
             [-5, -10, 0],
             [-2, -31, 0],
             [8, -20, 0],
             [15, -24, 0],
             [10, -9, 0],
             [14, -5, 0],
             [21, 1, 0],
             [5, 3, 0],
             [8, 16, 0],
             [0, 18, 0],
             [13, -29, 0],
             [7, -6, 0],
             [14, -9, 0],
 

In [3]:
from basic_usage.sketchformer import continuous_embeddings
sketchformer = continuous_embeddings.get_pretrained_model()


[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[run-experiment] resorting checkpoint if exists
[Checkpoint] Restored, step #207536


In [14]:
def convert_dataset_to_xy(dataset):
    input_batch = []
    label_batch = []
    for scene in dataset:
        sketches = list(map(lambda o: o['sketch'], scene))
        sketch_embeddings = sketchformer.get_embeddings(sketches)
        input_scene = []
        labels = []
        for se, obj in zip(sketch_embeddings, scene):
            p = [obj['position'][0] / 750, obj['position'][1] / 750]
            o = se.numpy().tolist() + p
            input_scene.append(o)  # オブジェクトの数が不規則
            labels.append(obj['label']) # TODO: convert to one-hot
        input_batch.append(input_scene)
        label_batch.append(labels)
    return tf.ragged.constant(input_batch).to_tensor(0.), tf.ragged.constant(label_batch).to_tensor('none')


In [15]:
x, y = convert_dataset_to_xy(sample)
print(x.shape)
print(y.shape)


[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (22, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (9, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (11, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (10, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extract-embeddings] batch_x shape (7, 200, 5)
[INFO] Loaded megabatch from /home/george/tanigaki/i-sketcher/interactive-sketcher/sketchformer/basic_usage/tmp_data/test.npz
[extra

In [7]:
from isketcher import InteractiveSketcher

interactive_sketcher_test = InteractiveSketcher(
    num_layers=2, d_model=130, num_heads=5, dff=2048,
    object_num=40, pe_target=40)

c_out, x_out, y_out, _ = interactive_sketcher_test(
    x, training=False, look_ahead_mask=None)

print(c_out.shape, x_out.shape, y_out.shape)
print(c_out) # class (one-hot)
print(x_out) # x (0 ~ 1)
print(y_out) # y (0 ~ 1)


(16, 26, 40) (16, 26) (16, 26)
tf.Tensor(
[[[0.01136152 0.00445375 0.03110466 ... 0.06557076 0.06398541 0.00940892]
  [0.00868003 0.0069221  0.03029007 ... 0.07706792 0.01717094 0.00554792]
  [0.00056418 0.00890182 0.01052878 ... 0.02031008 0.00201788 0.01750806]
  ...
  [0.01376773 0.0323501  0.00612547 ... 0.00970986 0.00056413 0.00254035]
  [0.01410213 0.02415517 0.0062928  ... 0.00969408 0.00057434 0.00272207]
  [0.0123192  0.01469923 0.00580978 ... 0.01159463 0.00150359 0.00148468]]

 [[0.01261186 0.02013204 0.00858359 ... 0.00741979 0.0034781  0.02241365]
  [0.00324815 0.00835347 0.03236841 ... 0.06228631 0.00156984 0.00465809]
  [0.02813735 0.03618304 0.01173275 ... 0.0064998  0.02057071 0.00425553]
  ...
  [0.01364401 0.00777653 0.02396191 ... 0.03208029 0.00721163 0.00389997]
  [0.01341871 0.00737774 0.02265902 ... 0.02996468 0.00726219 0.00370261]
  [0.00800802 0.00888717 0.01995277 ... 0.07046681 0.00193247 0.00749388]]

 [[0.0020692  0.00308438 0.0132878  ... 0.00164288 0.0

In [7]:
# Hyper parameters

num_layers = 4
d_model = 131
dff = 512
num_heads = 1

target_object_num = 40  # object num
dropout_rate = 0.1


In [8]:
# Optimizer

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)


In [9]:
# Loss function

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [10]:
interactive_sketcher = InteractiveSketcher(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
    target_vocab_size=target_object_num, pe_target=target_object_num, rate=dropout_rate)


In [14]:
# Create masks

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # アテンション・ロジットにパディングを追加するため
    # さらに次元を追加する
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_masks(inp, tar):
    # Encoderパディング・マスク
    enc_padding_mask = create_padding_mask(inp)

    # デコーダーの 2つ目のアテンション・ブロックで使用
    # このパディング・マスクはエンコーダーの出力をマスクするのに使用
    dec_padding_mask = create_padding_mask(inp)

    # デコーダーの 1つ目のアテンション・ブロックで使用
    # デコーダーが受け取った入力のパディングと将来のトークンをマスクするのに使用
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


def create_combined_mask(tar):

    # デコーダーの 1つ目のアテンション・ブロックで使用
    # デコーダーが受け取った入力のパディングと将来のトークンをマスクするのに使用
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return combined_mask


In [11]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=interactive_sketcher,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# チェックポイントが存在したなら、最後のチェックポイントを復元
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')


In [ ]:
# Training step

EPOCHS = 20

# @tf.functionは高速に実行するためにtrain_stepをTFグラフにトレースコンパイルします。
# この関数は、引数となるテンソルのshapeに特化したものです。
# シーケンスの長さや（最後のバッチが小さくなるなど）バッチサイズが可変となることによって
# 再トレーシングが起きないようにするため、input_signatureを使って、より一般的なshapeを
# 指定します。

train_step_signature = [
    tf.TensorSpec(shape=(None, None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    combined_mask = create_combined_mask(tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = interactive_sketcher(tar_inp, True, combined_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, interactive_sketcher.trainable_variables)
    optimizer.apply_gradients(
        zip(gradients, interactive_sketcher.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)


In [ ]:
# Training

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    # for (batch, (inp, tar)) in enumerate(train_dataset):
    #   train_step(inp, tar)

    #   if batch % 50 == 0:
    #     print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
    #         epoch + 1, batch, train_loss.result(), train_accuracy.result()))

    # とりあえず1epoch-1batchでやってみる
    train_step(x)
    print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(
        epoch + 1, train_loss.result(), train_accuracy.result()))

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                            ckpt_save_path))

    print('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1,
                                                        train_loss.result(),
                                                        train_accuracy.result()))

    print('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))
